In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import os
import calendar

In [2]:
#paths to kiki and will data 
k_disc_path = r"C:\Users\willd\OneDrive\Desktop\Budget\statements\kiki_disc"
k_chase_path = r"C:\Users\willd\OneDrive\Desktop\Budget\statements\kiki_chase"
k_ally_path = r"C:\Users\willd\OneDrive\Desktop\Budget\statements\kiki_ally"
# k_end_path = r"C:\Users\willd\OneDrive\Desktop\Budget\clean_data\k_data\k_budget_data.xlsx"
w_pnc_path = r"C:\Users\willd\OneDrive\Desktop\Budget\statements\pnc_csvs"
w_chase_path = r"C:\Users\willd\OneDrive\Desktop\Budget\statements\chase_csvs"
end_path = r"C:\Users\willd\OneDrive\Desktop\Budget\clean_data\combined_budget_data.xlsx"

## K Functions

In [3]:
# combine all csvs in a path return df  // with no duplicates 

def combine_csvs_pnc(path):
    df_list = []
    for item in os.listdir(path):
        full_path = os.path.join(path, item)
        df = pd.read_csv(full_path)
        df_list.append(df)
    combined = pd.concat(df_list)
    combined.columns.values[0] = "Date"
    no_dup = combined.drop_duplicates(subset=['Date', 'Description'], keep='first', inplace=False, ignore_index=True)
    return no_dup



def combine_csvs(path):
    df_list = []
    for item in os.listdir(path):
        full_path = os.path.join(path, item)
        df = pd.read_csv(full_path)
        df_list.append(df)
    combined = pd.concat(df_list, ignore_index=True)
#     combined = combined.drop_duplicates(keep='first')
    combined.columns.values[0] = "Date"
    no_dup = combined.drop_duplicates(keep='first', inplace=False, ignore_index=True)
    return no_dup


# remove specific payments from a df
def remove_item(df, descript):
    remove_df = df[~df['Description'].str.contains(descript)]
    return remove_df
    
    
# reformat to only date amount desc, category columns
def column_remove(df, user, bank):
    df.columns = df.columns.str.replace(' ', '') 
    if 'Category' not in df.columns:
        df["Category"] = 'misc'
    df["User"] = user
    df["Bank"] = bank
#     df["In/Out"] = 'tbd'
    new = df[['Date', 'Amount', 'Description', 'Category',"User","Bank"]]
    return new

# assign transaction catagory
def k_category_assign(df):
    # make seriess 
    s1 = df['Description']
    s2 = df['Amount']
    #ks category conditions and values
    k_cat_condition = [(s1.str.contains('VENMO'))&(s2 < (-600)),(s1.str.contains('VENMO')),
                       (s1.str.contains('COMCAST')),
                       (s1.str.contains('Primco')), (s1.str.contains('SCHWAB')), (s1.str.contains('Interest Paid'))]
    k_cat_value = ['RENT','VENMO','Bills & Utilities', 'Prim and Co', 'Investment', 'Intrest']
    df['Category'] = np.select(k_cat_condition, k_cat_value)
    df.loc[df['Category'] == '0', 'Category'] = 'Misc'
    return df 
def w_category_assign(w_pnc):
    # make seriess 
    df = w_pnc['Description']
    df2 = w_pnc['Amount']
    cat_condition = [(df.str.contains('VENMO')) & (df2 < 600),(df.str.contains('VENMO'))& (df2 > 0) & (df2 > 600), (df.str.contains('SPOTIFY') | df.str.contains('APPLE')),
                     (df.str.contains('ACTALENT')), (df.str.contains('CHECK'))
                    , (df.str.contains('SCHWAB BROK'))]
    cat_value = ['VENMO','KIKI RENT', 'SUBSCRIP', 'Belair', 'W_RENT', 'SCHWAB']
    w_pnc['Category'] = np.select(cat_condition, cat_value)
    return w_pnc

# , (df.str.contains('VENMO')) & (df2 < 0), 'VENMO OUT',  & (df2 > 0)

# function to fill ammount column in the pnc dataframe 
def amount_fill(data):
    if '$' in str(data['Withdrawals']):
        amount = '-' + data['Withdrawals']
    else:
        amount = data['Deposits']
    return float(amount.replace('$', '').replace(',', ''))

In [17]:
#compile all files in each folder
w_pnc = combine_csvs_pnc(w_pnc_path)
#make one row of just amount from withdrawls and eposits
w_pnc['Amount'] = w_pnc.apply(amount_fill, axis=1)
w_pnc = column_remove(w_pnc, 'W', 'PNC')
w_pnc = w_pnc.drop_duplicates()
w_chase = column_remove(combine_csvs(w_chase_path), 'W', 'chase')
# add catacories to pnc data frame
w_pnc = w_category_assign(w_pnc)
# make a misc catagory of remaining  
w_pnc.loc[w_pnc['Category'] == '0', 'Category'] = 'Misc'
# remove CC Payments from both 
w_pnc = w_pnc[~w_pnc['Description'].str.contains('CHASE CREDIT CRD EPA')]
w_chase = w_chase[w_chase['Amount'] < 0]
# remove pnc transfers 
w_pnc = w_pnc[~w_pnc['Description'].str.contains('ONLINE TRANSFER')]

k_disc = column_remove(combine_csvs(k_disc_path), 'K', 'disc')
k_chase = column_remove(combine_csvs(k_chase_path), 'K', 'chase')
k_ally = column_remove(combine_csvs(k_ally_path), 'K', 'ally')
# categorize ally transactions
k_ally = k_category_assign(k_ally)
#remove cc payments from ally
k_ally = remove_item(k_ally, 'CHASE CREDIT CRD')
k_chase = remove_item(k_chase, 'AUTOMATIC PAYMENT')
k_disc = remove_item(k_disc, 'DIRECTPAY FULL BALANCESEE DETAILS')
# make remoining discover 
k_disc['Amount'] = 0 - k_disc['Amount']

In [18]:
# combine pnc and chase for analysis 
# df_list = [w_pnc, w_chase, k_disc, k_chase,k_ally]
full_df = pd.concat([w_chase, k_disc, k_chase,k_ally, w_pnc])

# change date dtype
full_df['Date'] = pd.to_datetime(full_df['Date'])
# add month column 
full_df['Year'] = pd.DatetimeIndex(full_df['Date']).year
# add month column 
full_df['Month'] = pd.DatetimeIndex(full_df['Date']).month
# change month colum to date
full_df['Month'] = full_df['Month'].apply(lambda x: calendar.month_abbr[x])
full_df = full_df.sort_values(by='Date')

month_list = full_df['Month'].unique()
for mon1 in month_list:
    month_df = full_df[full_df['Month'] == mon1]
    k_rent = month_df.loc[month_df['Category'] == 'KIKI RENT', 'Amount']
    w_rent = month_df.loc[month_df['Category'] == 'W_RENT', 'Amount']  
    if (len(k_rent) > 0) & (len(w_rent) > 0):
        w_index = w_rent.index
        w_index = w_index[0]
        rent_actual = w_rent.iloc[0] + k_rent.iloc[0]
        full_df.loc[(full_df['Category'] == 'RENT')&(full_df['Month'] == mon1), 'Amount'] = rent_actual
full_df = full_df[full_df['Category'] != 'KIKI RENT']

full_df.loc[full_df['Category'] == "W_RENT", "Category"] = 'RENT'

# dfs of income and expenses 
money_in = full_df[full_df['Amount'] > 0]
money_out = full_df[full_df['Amount'] < 0]
money_out.loc[money_out['Amount'] < 0, 'Amount'] = 0 - money_out['Amount']


# df of expense to income per month 
month_list = full_df['Month'].unique()
dict_list = []
for mon in month_list:
    df_in = money_in[money_in['Month'] == mon]
    df_out = money_out[money_out['Month'] == mon]
    month_in = df_in['Amount'].sum()
    month_out = df_out['Amount'].sum()
    dict1 = {'Month': mon, 'Money In': month_in, 'Money Out': month_out}
    dict_list.append(dict1)
monthly_ratio = pd.DataFrame(dict_list)



C:\Users\willd\AppData\Local\Temp\ipykernel_6420\3007881497.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  money_out.loc[money_out['Amount'] < 0, 'Amount'] = 0 - money_out['Amount']


In [34]:
# full_df[full_df['User'] == 'K']

### Export to csv

In [26]:
writer = pd.ExcelWriter(end_path, engine="xlsxwriter")
full_df.to_excel(writer, sheet_name="full_df")
monthly_ratio.to_excel(writer, sheet_name="all_mon")
money_in.to_excel(writer, sheet_name="income")
money_out.to_excel(writer, sheet_name="expense")
writer.close()

In [27]:
full_df

,Date,Amount,Description,Category,User,Bank,Year,Month
698,2022-01-28,-36.26,TWISTED MINDS,Shopping,W,chase,2022,Jan
696,2022-01-30,-49.46,SHELL OIL 574243079QPS,Gas,W,chase,2022,Jan
697,2022-01-30,-105.34,GIANT-EAGLE #0208,Groceries,W,chase,2022,Jan
695,2022-02-01,-29.27,Amazon.com*350F49JK3,Shopping,W,chase,2022,Feb
694,2022-02-01,-13.81,CIRCLE K # 05700,Gas,W,chase,2022,Feb
...,...,...,...,...,...,...,...,...
61,2024-01-26,-11.87,RECURRING DEBIT CARD XXXXX9026 SPOTIFY ...,SUBSCRIP,W,PNC,2024,Jan
0,2024-01-26,-105.96,XCEL EZ-PAY WEB,Bills & Utilities,W,chase,2024,Jan
338,2024-01-28,-68.10,UBER TRIP,Travel,K,chase,2024,Jan
132,2024-01-29,-19.43,VENMO PAYMENT,VENMO,K,ally,2024,Jan


In [20]:
# # ws categories conditions and values
# w_cat_condition = [(df.str.contains('VENMO')) & (df2 > 0) & (df2 < 600),(df.str.contains('VENMO'))& (df2 > 0) & (df2 > 600), (df.str.contains('VENMO')) & (df2 < 0), (df.str.contains('SPOTIFY') | df.str.contains('APPLE')),
#                  (df.str.contains('ACTALENT')), (df.str.contains('CHECK'))
#                 , (df.str.contains('SCHWAB BROK'))]
# w_cat_value = ['VENMO IN','KIKI RENT','VENMO OUT', 'SUBSCRIP', 'Belair', 'RENT', 'SCHWAB']
# s1 = df['Description']
# s2 = df['Amount']
# k_cat_condition = [(s1.str.contains('VENMO'))& (s2 < 600),(s1.str.contains('VENMO'))&(s2 < -600),
#                    (s1.str.contains('COMCAST')),
#                    (s1.str.contains('Primco')), (s1.str.contains('SCHWAB')), (s1.str.contains('Interest Paid'))]
# k_cat_value = ['VENMO','RENT', 'Bills & Utilities', 'Prim and Co', 'Investment', 'Intrest']

In [24]:
ven = full_df[full_df['Category']== 'VENMO']
w_ven = ven[ven['User'] == 'W']
k_ven = ven[ven['User'] == 'K']

In [8]:
for index , row in w_ven.iterrows():
    k_ven
    print(row['Amount'])

-860.75
-899.92
-320.82
-115.54
-142.57


In [20]:
month_list = full_df['Month'].unique()
for mon1 in month_list:
    month_df = full_df[full_df['Month'] == mon1]
    ven = month_df[month_df['Category']== 'VENMO']
    w_ven = ven[ven['User'] == 'W']
    k_ven = ven[ven['User'] == 'K']
    for index , row in w_ven.iterrows():
        for k_index , k_row in k_ven.iterrows():
            zero = row['Amount'] + k_row['Amount']
            if zero == 0:
                print(k_index, index)

18 55


In [21]:
w_ven

,Date,Amount,Description,Category,User,Bank,Year,Month
6,2023-12-07,-115.54,ACH WEB XXXXX1578 VENMO PAY...,VENMO,W,PNC,2023,Dec


In [8]:

full_df =  pd.concat([disc_no_cc, chase_no_cc, ally_no_cc])

# change date dtype
full_df['Date'] = pd.to_datetime(full_df['Date'])
full_df['Month'] = pd.DatetimeIndex(full_df['Date']).month
full_df = full_df.sort_values(by='Date')

full_df = full_df[full_df['Date'] >= '2023-09-01']
# change month colum to date
full_df['Month'] = full_df['Month'].apply(lambda x: calendar.month_abbr[x])
full_df = full_df.sort_values(by='Date')

full_df = full_df.reset_index(drop=True)

full_df.loc[full_df['Amount'] < 0, "In/Out"] = 'out'
full_df.loc[full_df['Amount'] > 0, "In/Out"] = 'in'
full_df.loc[full_df['Amount'] < 0, 'Amount'] = 0 - full_df['Amount']

In [28]:
# writer = pd.ExcelWriter(end_path, engine="xlsxwriter")
# full_df.to_excel(writer, sheet_name="full_df")
# # monthly_ratio.to_excel(writer, sheet_name="all_mon")
# # money_in.to_excel(writer, sheet_name="income")
# # money_out.to_excel(writer, sheet_name="expense")
# writer.close()